<h1 style="text-align: center;"><b>Modern Topics in Interest Rates Modelling</b></h1>

### Roland Grinis


In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("dark_background")
%matplotlib inline

In [ ]:
from pyquant.torch_spline import CubicSpline1D

In [ ]:
#data
fwd_ois = pd.read_csv('data/forward_ois.csv')
fwd_key_rate = pd.read_csv('data/forward_key_rate.csv')
vol_key_rate = pd.read_csv('data/volatility_key_rate.csv')

## 1. Risk Free Rates (RFRs)

### 1.1 Extended $T$-forward measure

Short (risk-free, OIS) rate $r_t$ gives rise to bank account numeraire:

$$
dB_t = r_t B_t dt, \quad B_t = \exp \left( \int_0^t r_u du \right)
$$

The associcated risk neutral measure $Q$ is called the money-savings measure
Investing in the bank account after maturity, we obtain the extended zero-coupon: 
$$
P_{t,T} = 
\begin{cases}
  \mathbb{E}^Q_t \left[ \exp \left(- \int_{t}^T r_u du \right) \right] , & t\leq T\\
  \exp \left(- \int_{t}^T r_u du \right) = \frac{B_t}{B_T},            & t > T
\end{cases}
$$

The associated risk-neutral measure, denoted $Q^T$, corresponds to the $T$-forward measure for $t \leq T$
and the money-savings measure $Q$ after maturity.   

### 1.2 Backward-looking forward rates

Forward Rate Agreement (FRA) contract has present value:

$$
PV_t = B_t \cdot \mathbb{E}^Q_{t} \left[ \tau \frac{\mathcal{P}_{T-\tau, T} - K}{B_T}  \right]
$$

with forward looking payoff:

$$
\mathcal{P}_{T-\tau, T} \equiv \mathcal{P}_{T-\tau, T} \left[ T' \rightarrow P_{T-\tau, T'} \right]
$$

or backward looking payoff:
$$
\mathcal{P}_{T-\tau, T} \equiv \mathcal{P}_{T-\tau, T} \left[ r_d \in \mathcal{B}(T - \tau, T) \right]
$$
where $\mathcal{B}(T - \tau, T)$ runs over business days in $\left[ T - \tau, T \right)$.

For example, the simple daily-compounded setting-in-arrears rates read:
$$
R_T \coloneqq R(T-\tau, T) = \mathcal{P}_{T-\tau, T} = \frac{1}{\tau} \left( \prod_{t \leq t_n < t+\tau} \left(1 + \tau_n r_{t_n} \right) - 1 \right)
$$

Or the daily arithmetic average setting-in-arrears rates read:

$$
A(T - \tau, T) = \mathcal{P}_{T-\tau, T} = \frac{1}{\tau} \sum_{t \leq t_n < t+\tau} \tau_n r_{t_n} 
$$

Passing to continous modelling:
$$
\frac{1}{\tau} \left[ \exp \left( \int_{T - \tau}^T r_u du \right)  - 1\right], \quad A(T - \tau, T)\approx \frac{1}{\tau} \int_{T - \tau}^T r_u du 
$$

and we get the relation:
$$
A(T - \tau, T) \approx \frac{1}{\tau} \log \left(1 + \tau R(T - \tau, T)\right)
$$



Passing to the extended $T$-forward measure, we obtain the compound forward rate as:
$$
R_{t,T} \coloneqq R_t(T - \tau, T) = \mathbb{E}^{Q^T}_{t} \left[  R(T - \tau, T) \right]
$$

which make $PV_t = 0$. We can evaluate the expectation:


* for $t \leq T-\tau$, $ R_{t,T} = \frac{1}{\tau} \left( \frac{P_{t, T - \tau}}{P_{t,T}} - 1\right)$ is analogous to IBOR forward rates.
* for $ T-\tau < t \leq T$, $ R_{t,T} = \frac{1}{\tau} \left( \frac{B_t}{B_{T - \tau} P_{t,T}} - 1\right)$ is daily compounding.
* post maturity $ R_{t,T} \equiv \frac{1}{\tau} \left( \frac{B_{T}}{B_{T - \tau}}  - 1\right)$ is known and fixed.

  
The yield curve $T \rightarrow P_{t,T}$ can be boostrapped from $T \rightarrow  R_{t,T}$ quotes.

The arithmetic average forward rate can be expressed as:
$$
A_{t,T} \coloneqq  A_t(T - \tau, T) = \mathbb{E}^{Q^T}_{t} \left[ \frac{1}{\tau} \log \left(1 + \tau R(T - \tau, T)\right) \right]
$$
and since the payoff is non-linear, $A_{t,T}$ is model dependent. A convexity correction can appear compared to market quotes:
$$
A_{t,T} = \frac{1}{\tau} \log \left(1 + \tau R_{t,T}\right) + \text{Convexity Correction}
$$

#### 1.2.1 Interest Rate Swap
Interest Rate Swap (IRS) based on simple daily-compounding backward rates values:
$$
PV_t = \sum_{i=1}^n \tau_i P_{t,T_i}R_t(T_{i-1}, T_i) - K \cdot  \sum_{j=1}^m \tau'_i P_{t, T'_j}
$$
with floating leg tenors $T_0, \dots, T_n$ and fixed leg $T'_0, \dots, T'_m$ tenors. 

For $t < T_0$ forward swap rate with associated present value of a basis point evaluates:
$$
S_t(T_0, T_n) = \frac{P_{t,T_0} - P_{t,T_n}}{\text{PVBP}(t)} , \quad \text{PVBP}(t) = \sum_{j=1}^m \tau'_j P_{t, T'_j}
$$

Consider the OIS FRA quotes:

In [ ]:
fwd_ois.set_index('tenor').sort_index().forward_rate.plot(title='Forward OIS Rate', grid=True, figsize=(20,10));

Compute the yield curve $T \rightarrow P_{t,T}$  using the above quotes.

In [ ]:
def build_ois_yield_curve_from_now_starting(forwards, tenors):
    zcbs = 1 / (1 + tenors * forwards)
    return CubicSpline1D(tenors, zcbs)

ois_yield_curve = build_ois_yield_curve_from_now_starting(
    torch.tensor(fwd_ois.forward_rate.values),
    torch.tensor(fwd_ois.tenor.values)
)

Let's compute IRS swap rates with tenors ranging from 1M to 10Y starting overnight, to bootstrap the yield curve $P_{t,T}$ back. In fact, the swap rate formula leads to a linear equation:
$$
P_{t,T_m} + \tau S_t(T_{1,\dots,m}) \sum_{j=1}^m P_{t, T_j} = 1
$$
where the coupons $P_{t,T_i}$ are variables and different swap quotes $S_t(  T_{m, \dots, n})$ lead to a system of linear equations that can be solved using the SVD method for $P_{t,T_i}$. One can then fit a polynomial model to smooth out the yield curve.

In [ ]:
tau = 1/12
n_coupons = 120
maturities = tau * torch.ones(n_coupons).cumsum(0)
num_months = 1 + torch.arange(0, 120, step=1)
n_tenors = num_months.numel()
A = torch.zeros((n_tenors,120))
for i in range(n_tenors):
    n_months = num_months[i].item()
    payment_dates = (1 + torch.arange(n_months)) / 12
    zcb_last = ois_yield_curve.evaluate(payment_dates[-1]).sum()
    tau_swap_rate = (1 - zcb_last) / ois_yield_curve.evaluate(payment_dates).sum()
    for j in range(n_months):
        A[i][j] = tau_swap_rate.item()
    A[i][n_months-1] += 1
U, S, Vh = torch.linalg.svd(A)

In [ ]:
bootstrap_zcb = torch.mv(Vh[:n_tenors].t(), torch.mv(U.t(), torch.ones(n_tenors)) / S)

In [ ]:
pd.DataFrame({
    'maturities' : maturities.numpy(),
    'zcbs': ois_yield_curve.evaluate(maturities).flatten().numpy(),
    'bootstrap': bootstrap_zcb.numpy()}).set_index('maturities').plot(legend=True, grid=True, figsize=(20,10));

### 1.3 Forward Market Model (FMM)
As we approach maturity $T$, backward-looking rates settle to constant, and we need to choose a cutoff function to dampen volatility. 
  
A typical example is the linear cutoff $g_{t,T} = (T-t)^+ / \tau $ for $t \in (T-\tau, T)$ with $g_{t,T} \equiv 1$ for $t \leq T-\tau$ and $g_{t,T} \equiv 0$, $t \geq T$.  
  
In extended $T$-forward measure:
$$
dR_{t,T} = g_{t,T} \sigma_{t,T} (R_{t,T}) dW_t^{Q^T} \quad \text{martingale,}
$$

where $\sigma_{t,T} \equiv \sigma^T$ (Bachelier), or $\sigma_{t,T} \equiv \sigma^T R_{t,T}$ (Black), or  $\sigma_{t,T} \equiv \sigma^T (R_{t,T})^{\beta}$ (CEV), or Dupire local vol, or stochastic vol etc. with (multi-dimensional) Brownian motion $W_{t,T}$

In money-savings numeraire $B_t$, we get the drift:
$$
dR_{t,T} = g_{t,T} \sigma_{t,T}(R_{t,T}) \left[
\sum_{T' < T}\rho_{T',T} \frac{\tau' g_{t,T'} \sigma_{t,T'}(R_{t,T'})}{1 + \tau' R_{t,T'} } dt + dW_t^Q
\right]
$$
where the sum runs over shorter tenors and $\rho$ holds the correlation structure of the curve.

#### 1.3.1 Caplet Pricing

Using $P_{t,T}$ as numeraire, for simply-compound rates:

$$
V^{\text{Caplet}}_t \left[ R_{t,T}, K \right] = P_{t,T} \mathbb{E}_{t}^{Q^T} \left[ \tau \left( R(T-\tau, T) - K \right)^+ \right] 
$$

For arithmetic average compound rates, the choice of cutoff $g_{t,T}$ influence directly the convexity correction. 
One should attempt to choose it such that we have:
$$
V^{\text{Caplet}}_t \left[ A_{t,T}, K \right] = P_{t,T} \mathbb{E}_{t}^{Q^T} \left[ \tau \left( A(T-\tau, T) - K \right)^+ \right] =  P_{t,T} \mathbb{E}_{t}^{Q^T} \left[  \left( \log \left(1 + \tau R(T - \tau, T)\right)  - \tau K \right)^+ \right] 
$$ 

As reference, using the linear cutoff:
$$
\hat{g}_{t,T}^2 \coloneqq \int_{t,T} g_{t,T}^2(s)ds = 
\begin{cases}
  T - t - \frac{2}{3}\tau,  & t\leq T-\tau \\
  \frac{(T-t)^3}{3\tau^2},  & t > T - \tau
\end{cases}
$$

Bachelier model:
$$
dR_{t,T} = g_{t,T} \sigma dW_t^{Q^T}, \quad R(T-\tau, T) \sim \mathcal{N}\left(R_{t,T}, (\sigma\hat{g}_{t,T})^2 \right) 
$$ 


Black model:
$$
dR_{t,T} = g_{t,T} R_{t,T} \sigma dW_t^{Q^T}, \quad R(T-\tau, T) = R_{t,T} \exp \left( - \frac{(\sigma\hat{g}_{t,T})^2 }{2} + \mathcal{N}\left(0, (\sigma\hat{g}_{t,T})^2 \right)  \right)
$$

Market makers formula convention for arithmetic-average compound rates:
$$
V^{\text{Caplet}}_t \left[ A'_{t,T}, K \right] = \tau P_{t,T}\left[ \left( A'_{t,T} - K\right) \Phi\left(\frac{A'_{t,T} - K}{\sigma \hat{g}_{t,T} }\right) + \sigma \hat{g}_{t,T} \phi\left(\frac{A'_{t,T} - K}{\sigma \hat{g}_{t,T} }\right)\right]
$$
which is the Bachelier model (normal volatility, $\Phi$ and $\phi$ are normal distribution CDF and PDF respectively), or:
$$
V^{\text{Caplet}}_t \left[ A'_{t,T}, K \right] = \tau P_{t,T}\left[ A'_{t,T} \Phi\left(d_{+}\right) - K \Phi \left(d_{-} \right) \right], \quad d_{\pm} = \frac{\log (A'_{t,T} / K) \pm (\sigma \hat{g}_{t,T})^2 / 2 }{\sigma\hat{g}_{t,T}}
$$
with the Black model (log-normal volatility). No convexity correction is taken into account. However, the discounting $P_{t,T}$ is done with respect to the OIS curve, even if  $A'_{t,T}$ correspond to a different RFR rate (e.g. a Key Rate). 

In [ ]:
fwd_key_rate.set_index('tenor').sort_index().forward_rate.plot(title='Forward Key Rate', grid=True, figsize=(20,10));

In [ ]:
for k,v in vol_key_rate.groupby('time_to_maturity'):
    v.set_index('strike').sort_index().implied_normal_vol.rename(f'tenor {round(k,2)}').plot(legend=True, grid=True, figsize=(20,10));

In [ ]:
def key_rate_fwd_curve_from_now_starting(forwards, time_to_maturity):
    return CubicSpline1D(time_to_maturity, forwards)

def caplet_premium_from_now_starting(vol_surface, key_rate_fwd, ois_yield_curve):
    time_to_maturity = torch.tensor(vol_surface.time_to_maturity.values)
    tenor = torch.tensor(vol_surface.tenor.values)
    g_hat = torch.sqrt(time_to_maturity - (2/3) * tenor)

    strike = torch.tensor(vol_surface.strike.values)
    fwd = key_rate_fwd.evaluate(time_to_maturity).flatten()
    discount = ois_yield_curve.evaluate(time_to_maturity).flatten()
    iv = torch.tensor(vol_surface.implied_normal_vol.values)

    sigma_moneyness = (fwd - strike) / (iv * g_hat)

    normal = torch.distributions.Normal(0,1)
    Phi = normal.cdf
    log_phi = normal.log_prob

    pv = tenor * discount * ( (fwd - strike) * Phi(sigma_moneyness) + iv * g_hat * torch.exp(log_phi(sigma_moneyness)) )
    return torch.maximum(pv,torch.tensor(0.))

In [ ]:
key_rate_fwd_curve = key_rate_fwd_curve_from_now_starting(
    torch.tensor(fwd_key_rate.forward_rate.values),
    torch.tensor(fwd_key_rate.time_to_maturity.values)
)

In [ ]:
vol_key_rate['pv'] = caplet_premium_from_now_starting(vol_key_rate, key_rate_fwd_curve, ois_yield_curve).numpy()

In [ ]:
for k,v in vol_key_rate.groupby('time_to_maturity'):
    v.set_index('strike').sort_index().pv.rename(f'tenor {round(k,2)}').plot(legend=True, grid=True, figsize=(20,10));

### 1.4 Short Rate Models: Hull-White

Dynamics in money-savings risk-neutral measure $Q$ are given by:
$$
dr_t = \left( \theta_t - \lambda r_t \right) dt + \sigma dW_t
$$

with $\theta_t$ fixed by the HJM no-arbitrage condition. This can be rewritten as affine process:

$$
r_t = \alpha_t + x_t 
$$

where the deterministic part follows the HJM condition:
$$
\alpha_T = - \frac{\partial}{ \partial T}\log P_{t,T} + \frac{\sigma^2}{2 \lambda^2} \left( 1- e^{\lambda (T-t)} \right)
$$
and the stochastic part:
$$
dx_t = - \lambda x_t dt + \sigma dW_t
$$
which is simply a Gaussian:
$$
x_T \sim \mathcal{N} \left( x_t e^{-\lambda(T-t)}, \quad \frac{\sigma^2}{2 \lambda} \left( 1- e^{-2\lambda (T-t)} \right) \right)
$$

Changing to the $T$-forward measure $Q^T$: 
$$
dx_t = - \left( \frac{\sigma^2}{\lambda} \left(1 - e^{-\lambda(T-t)} \right) + \lambda x_t \right) dt + \sigma dW^T_t
$$

which is also a gaussian with mean:
$$
\mathbb{E}^{Q^T}_{t} \left[ x_{t+\Delta} \right] = x_t e^{-\lambda \Delta} - \frac{\sigma^2}{\lambda^2} \left(1 - e^{-\lambda \Delta} \right) - \frac{\sigma^2}{2\lambda^2} \left(e^{-\lambda(T-t-\Delta)} - e^{-\lambda(T-t+\Delta)} \right) 
$$
and variance $\frac{\sigma^2}{2 \lambda} \left( 1- e^{-2\lambda \Delta} \right)$ as in measure $Q$.

In [ ]:
timeline = torch.linspace(0, 10., 3651)[1:] # 10 years Actual/365 day

In [ ]:
def generate_hull_white(
        n_paths: int,
        n_steps: int,
        dt: torch.Tensor,
        init_state: torch.Tensor,
        lam: torch.Tensor,
        sigma: torch.Tensor
) -> torch.Tensor:
    
    paths = torch.empty((n_paths, n_steps + 1), dtype=init_state.dtype)
    paths[:, 0] = init_state
    for i in range(0, n_steps):
        x = paths[:, i].clone()
        Z = torch.randn_like(x)
        x_next = x * torch.exp(- dt * lam) + Z * torch.sqrt( sigma * sigma * (1 - torch.exp(- 2 * dt * lam)) / (2 * lam) ) 
        paths[:, i+1] = x_next
    return paths

In [ ]:
x0 = torch.tensor(0., requires_grad = True)
lam = torch.tensor(1., requires_grad = True)
sigma = torch.tensor(0.3, requires_grad = True) 

In [ ]:
hw_paths = generate_hull_white(1000, timeline.shape[0], timeline[0], x0, lam, sigma)

In [ ]:
pv = torch.maximum(hw_paths[:,-1] - 0. , torch.tensor(0.)).mean()

In [ ]:
grad = torch.autograd.grad(pv, [x0, lam, sigma])
grad

#### 1.4.1 The Hull-White-Heston model

To price the volatility smile we rely on stochastic volatility models such as the Heston Model, which combined with Hull-White looks like:

$$
\left\{
\begin{array}{l}
    dx_t = - \lambda x_t dt + \sqrt{v_t} \text{d}W_t^x,  \\
    dv_t =  \kappa (\theta - v_t)\text{d}t + \varepsilon \sqrt{v_t} \text{d} W_t^v,
\end{array}\right.
$$
with no correlation between the Brownian motions $dW_t^x \cdot dW_t^v = 0$, as this breaks the affinity of the process when changing from measure $Q$ to the $T$-forward $Q^T$: 
$$
\left\{
\begin{array}{l}
    dx_t =  - \left( \frac{v}{\lambda} \left(1 - e^{-\lambda(T-t)} \right) + \lambda x_t \right) dt + \sqrt{v_t} \text{d}W_t^x,  \\
    dv_t =  \kappa (\theta - v_t)\text{d}t + \varepsilon \sqrt{v_t} \text{d} W_t^v,
\end{array}\right.
$$

The short rate itself:
$$ 
r_t = \alpha_t + x_t 
$$ 
where the HJM condition on $\alpha_t$ is described in the multiple-curve framework below.

Stochastic variance is an CIR process with non-central chi-squared distribution:
$$
\mathbb{P}\left(v_{t + \Delta}<v \mid v_{t}\right) \quad = \quad F_{\chi'^{2}}\left(\frac{4 \kappa v}{\varepsilon^{2}\left(1-e^{-\kappa \Delta}\right)} ; \frac{4 \kappa \theta}{\varepsilon^{2}}, \frac{4 v_{t} \kappa e^{-\kappa \Delta}}{\varepsilon^{2}\left(1-e^{-\kappa \Delta}\right)}\right) \quad
$$
over a time step $\Delta$, and admits an almost exact QE simulation scheme.  

Since no correlation leaks can occur, the Euler scheme is sufficient to evolve $x_t$ and $v_t$:
$$
x_{t+\Delta} \sim \mathcal{N} \left( x_t e^{-\lambda \Delta}, \quad \frac{v_t}{2 \lambda} \left( 1- e^{-2\lambda \Delta} \right) \right)
$$
and similarly in $T$-forward measure.


In [ ]:
from pyquant.heston_sim import generate_cir

In [ ]:
v0 = torch.tensor(0.65, requires_grad = True)
theta = torch.tensor(1., requires_grad = True)
kappa = torch.tensor(1.3, requires_grad = True) 
epsilon = torch.tensor(1., requires_grad = True) 

In [ ]:
cir_paths = generate_cir(1000, timeline.shape[0], timeline[0], v0, kappa, theta, epsilon, 1e-6)

In [ ]:
def generate_hull_white_heston(
        n_paths: int,
        n_steps: int,
        dt: torch.Tensor,
        init_state: torch.Tensor,
        lam: torch.Tensor,
        var: torch.Tensor
) -> torch.Tensor:
    assert(var.shape == (n_paths, n_steps + 1))
    paths = torch.empty((n_paths, n_steps + 1), dtype=init_state.dtype)
    paths[:, 0] = init_state
    for i in range(0, n_steps):
        x = paths[:, i].clone()
        Z = torch.randn_like(x)
        sigma_2 = var[:, i]
        x_next = x * torch.exp(- dt * lam) + Z * torch.sqrt( sigma_2 * (1 - torch.exp(- 2 * dt * lam)) / (2 * lam) ) 
        paths[:, i+1] = x_next
    return paths

In [ ]:
x0 = torch.tensor(0., requires_grad = True)
lam = torch.tensor(1., requires_grad = True)

In [ ]:
hwh_paths = generate_hull_white_heston(1000, timeline.shape[0], timeline[0], x0, lam, cir_paths)

In [ ]:
pv = torch.maximum(hwh_paths[:,-1] - 0. , torch.tensor(0.)).mean()

In [ ]:
grad = torch.autograd.grad(pv, [x0, lam, v0, kappa, theta, epsilon])
grad

## 2. Multi-Curve models

### 2.1 Affine multi-curve models and IBOR rates

The short rate is given by:
$$
r_t = f(t) + \lambda^T \textbf{x}_t
$$
where $\textbf{x}_t$ is an affine process and $\lambda$ is constant. The latter drive also the instantaneous forward spread rates to other curves:
$$
s_t^{\delta} = s^{\delta}(t) + \gamma_{\delta}^T \textbf{x}_t
$$
enumerated by $\delta$, with $\gamma_{\delta}$ constant. If those spreads corresponds to IBOR rates, then $\delta$ are the tenors, and we require:
$$
0 \leq s_t^{\delta_i}  \leq s_t^{\delta_j}, \quad  \delta_i < \delta_j
$$
in which case we can calibrate to the available ZCB curves:
$$
T \rightarrow P_{t,T} = \mathbb{E}_t \left[ e^{- \int_{t,T} r_u du} \right], \quad P^\delta_{t,T} = \mathbb{E}_t \left[ e^{ - \int_{t,T} \left(r_u + s^{\delta}_u \right)du} \right]
$$

where $P_{t,T}$ is the OIS curve, and the other correspond to IBOR forward rates at different tenors:
$$
L_t(T-\delta, T)=  \frac{P_{t,T-\delta}^\delta - P_{t,T}^\delta}{\delta \cdot P_{t,T}^\delta} 
$$

Denoting the forward spreads:
$$
S^{\delta}_t(T) = \frac{1 + \delta \cdot L_t(T-\delta, T)}{1 + \delta \cdot L^D_t(T-\delta,T)} = \frac{P_{t,T-\delta}^\delta}{P_{t,T}^\delta} \frac{P_{t,T}}{P_{t,T-\delta}}
$$
where $L^D_t(T-\delta, T) =   \frac{P_{t,T-\delta} - P_{t,T}}{\delta P_{t,T} }$ is the forward looking RFR.

We recall that the instantaneous forward is given by:
$$
f_{t,T} = - \frac{\partial}{\partial T} \log P_{t,T}
$$

whereas the instantenous multiplicative spread:
$$
s^{\delta}_{t,T} =  \frac{\partial}{\partial T} \log S^\delta_t(T)
$$

note the difference is signs, as heuristically the instantenous forward for the tenor-$\delta$ curve is  $f_{t,T}^\delta = f_{t,T} + s^{\delta}_{t,T}$.

The HJM condition in that case reduces to:
$$
f(t) = f_{t,T} - \bar{f}_{t,T}, \quad s^{\delta}(t) = \frac{\partial}{\partial t} \log S^\delta _0(t) - \frac{\partial}{\partial t} \log \bar{S}^\delta _0(t) 
$$
where the base curves $\bar{P}_{0,t}$ and $\bar{S}^\delta _0(t)$ are obtained from a model where $f(t)\equiv 0$ and $s^{\delta}(t) \equiv 0$.


#### 2.1.1 Linear Products

IBOR quotes typically come from FRAs, whose present value at strike $K$ is given by:
$$
PV_t\left[K \right] = P_{t,T} S^{\delta}_t(T)- (1 + \delta K) P_{t, T+\delta}
$$

Using IRSs quotes, one solves for forward spreads $S^{\delta}_t(T)$ directly, as the former evaluates to:
$$
PV_t \left[\delta,  \delta', K\right] = \sum_{i \geq 1} \left( P_{t, T_{j-1}}  S^{\delta}_t(T_{j-1}) -  P_{t, T_{j}} \right) - \delta' K \sum_{j\geq 1} P_{t, T'_j}
$$
where $\delta = T_{i} - T_{i-1}$ is the floating leg tenor and $\delta' = T'_{j} - T'_{j-1}$ is the fixed leg tenor. 

#### 2.1.2 Model with CIR processes
A typical example can be obtained using self-exciting CIR processes for the forward spreads and Hull White dynamics for the rate:
$$
s_t^{\delta_j} = s^{\delta_j}(t) + \sum_{i \leq j} \gamma_i x_{t}^i, \quad \gamma_i \geq 0, \quad dx_{t}^i= \kappa_i (\theta_i - x_{t}^i) dt + \epsilon_i \sqrt{x_{t}^i} dW_t^i
$$
where tenors $\delta_{1} < \delta_{2} < \dots < \delta_{n}$ are in increasing order. 

The short rate  follows the Hull-White-Heston multi-factor model:
$$
r_t = f(t) + x_t^0, \quad dx_{t}^0 = - \lambda_0 x_{t}^0 dt + \sum_{i=1}^n \lambda_i \sqrt{x_{t}^i} dW_t^i
$$
where $\lambda_1^2 + \dots + \lambda_n^2 =1$, and all Brownian motions $W_t^i$ are independent.


### 2.2 Modelling Arithmetic-Average Compounding on a Key Rate

We are not confined to model IBOR rates with spreads $s_t^{\delta}$. In fact, we can choose any other curve, in which case we will need to fit whatever market quotes are available to us. 

In our case, we have a forward curve for arithmetic-average backward looking $A_{t,T}$ on a Key Rate, which is different from the OIS $r_t$ for which we also have forward curve. And we need to capture the smile of a volatility surface on $A_{t,T}$. 

We can add to the Hull-White-Heston model a stochastic spread between OIS short rate and the Key Rate:
$$
r_t = f(t) + x_t, \quad dx_{t} = - \lambda x_{t} dt + \sqrt{v_t} dW_t^x, \quad dv_t =  \kappa (\theta - v_t)dt + \varepsilon \sqrt{v_t} dW_t^v
$$
as above and the spread follows a mean-reversion processess near zero:
$$
s_t = s^a(t) + k_t, \quad dk_t =  - \gamma k_t dt + \xi dW_t^k
$$
with all Brownian motions $W_t^x, W_t^v$ and $W_t^k$ independent. 

Note that the spread $s^a(t)$ does not depend on a tenor $\tau$ as we are working with RFR rates, and therefore can be computed from any tenor. The most convenient is the one corresponding to time to maturity $\tau = T-t$, and in fact it is the one which is typically quoted. In that case:
$$
s^a(t) = \left( f_{t,T}^a - f_{t,T} \right) - \left( \bar{f}_{t,T}^a - \bar{f}_{t,T} \right)
$$
where the top-bar terms are the ones obtained from the model with $s^a(t) \equiv 0$ and $f(t) \equiv 0$ as above.

#### Exercise 2.2.1

Obtain the OIS instantaneous rates $f_{t,T}$ from:

In [ ]:
def build_fwd_curve_from_now_starting(forwards, tenors):
    zcbs = 1 / (1 + tenors * forwards)
    log_zcbs = torch.log(zcbs)
    return CubicSpline1D(tenors, - log_zcbs)

ois_fwd = build_fwd_curve_from_now_starting(
    torch.tensor(fwd_ois.forward_rate.values),
    torch.tensor(fwd_ois.tenor.values)
)

Ignoring the convexity correction, obtain the Key Rate instantaneous rates $f^a_{t,T}$ from:

In [ ]:
fwd_key_values = torch.tensor(fwd_key_rate.forward_rate.values, requires_grad=True)

In [ ]:
key_fwd = build_fwd_curve_from_now_starting(
    fwd_key_values,
    torch.tensor(fwd_key_rate.time_to_maturity.values)
)

Obtain the parameters for the factors $x_t$, $v_t$ and $k_t$ as to fit the Caplet prices:
$$
V^{\text{Caplet}}_t \left[ A_{t,T}, K \right] = B_t \mathbb{E}_{t}^{Q} \left[ \left( \int_{T - \tau}^T r_u + s_udu - \tau K \right)^+ / B_T\right] 
$$
from

and the Key Rate forwards using:
$$
A_{t,T} = \mathbb{E}^{Q^T}_{t} \left[ \frac{1}{\tau} \int_{T - \tau}^T r_u + s_u du \right] = \frac{ \mathbb{E}_{t}^{Q} \left[ \frac{1}{\tau} \int_{T - \tau}^T r_u + s_u du / B_T \right] }{\mathbb{E}_{t}^{Q} \left[ 1/B_T \right]}
$$
via Monte-Carlo.

In [ ]:
fwd_key_values

In [ ]:
params = torch.tensor(
    [
        0.65, #v0 - 0
        1., #kappa - 1
        1., #theta - 2
        1., #epsilon - 3
        1., #lam - 4
        1., #gamma - 5
        0.01, #xi - 6 
        
    ],
    requires_grad = True
)

In [ ]:
v_paths = generate_cir(1000, timeline.shape[0], timeline[0], params[0], params[1], params[2], params[3], 1e-6)

In [ ]:
x_paths = generate_hull_white_heston(1000, timeline.shape[0], timeline[0], torch.tensor(0.), params[4], cir_paths)

In [ ]:
ks_paths = generate_hull_white(1000, timeline.shape[0], timeline[0], torch.tensor(0.), params[5], params[6])

In [ ]:
ids = (torch.tensor(fwd_ois.time_to_maturity.values) * 365).ceil().int() - 1
dt = timeline[0]
zcb_ttm = timeline[ids]
zcbs_r = torch.exp( - dt * x_paths.cumsum(1)[:,ids]).mean(0)
r_fwd  = CubicSpline1D(zcb_ttm, - torch.log(zcbs_r))
zcbs_a = torch.exp( - dt * (x_paths + ks_paths).cumsum(1)[:,ids]).mean(0)
a_fwd =  CubicSpline1D(zcb_ttm, - torch.log(zcbs_a))

In [ ]:
f_curve = ois_fwd.derivative(timeline).T - r_fwd.derivative(timeline).T
s_curve = key_fwd.derivative(timeline).T - a_fwd.derivative(timeline).T - f_curve
r_paths = f_curve + x_paths[:,1:]
s_paths = s_curve + ks_paths[:,1:]

In [ ]:
int_r_paths = dt * r_paths.cumsum(1)
int_r_s_paths = int_r_paths + dt * s_paths.cumsum(1)

In [ ]:
vol_ids = (torch.tensor(vol_key_rate.time_to_maturity.values) * 365).ceil().int() - 1
tau_strikes = timeline[vol_ids] * torch.tensor(vol_key_rate.strike.values) 
market_pvs = torch.tensor(vol_key_rate.pv.values)
model_pvs = torch.zeros_like(market_pvs)

In [ ]:
for i in range(model_pvs.numel()):
    T = vol_ids[i]
    tauK = tau_strikes[i]
    tau = timeline[T]
    B_T = torch.exp(int_r_paths[:, T]) 
    payoff = torch.maximum(int_r_s_paths[:, T] - tauK, torch.tensor(0.)) 
    model_pvs[i] = torch.mean(payoff / B_T)

In [ ]:
key_fwd_ids = (torch.tensor(fwd_key_rate.time_to_maturity.values) * 365).ceil().int() - 1
market_key_fwd = torch.tensor(vol_key_rate.pv.values)
model_key_fwd = torch.zeros_like(market_key_fwd)

In [ ]:
for i in range(model_key_fwd.numel()):
    T = vol_ids[i]
    tau = timeline[T]
    B_T = torch.exp(int_r_paths[:, T]) 
    A_T = int_r_s_paths[:, T] / tau
    model_key_fwd[i] = torch.mean(A_T / B_T) / torch.mean(1 / B_T)

In [ ]:
error = 0.001*((model_pvs - market_pvs) ** 2).sum() + ((market_key_fwd - model_key_fwd) ** 2).sum()

In [ ]:
grad_error = torch.autograd.grad(error, [params, fwd_key_values])

In [ ]:
grad_error[0]

In [ ]:
grad_error[1]

## 3 Product examples

### 3.1 Swaptions

A physically-settled payer (+) or receiver (-) swaption at the start $T_0$ of the IRS swap expires with value:
$$
PV_{T_0} = \text{PVBP}(T_0) \left(\pm \left( S_{T_0} - K\right)\right)^+
$$
where the present value of basis point $\text{PVBP}(T_0) =  \delta' \sum_{i > 0} P_{T_0, T_i}$ is computed from the risk free OIS curve and the swap rate $S_{T_0}$ comes from the floating rate that might be backward looking or IBOR based.

Using $\text{PVBP}(t)$ as numeraire we can price the swaption:
$$
PV_t = \text{PVBP}(t) \cdot \mathbb{E}_{t}^{\text{PVBP}} \left[(\pm\left( S_{T_0} - K)\right)^+ \right] 
$$
in which the swap rate $S_t$ is naturally a martingale.

For cash-settled swaptions, the payoff is settled at $T_0$ according to the formula:
$$
PV_{T_0} = \text{Ann}(S_{T_0}) \left(\pm  \left( S_{T_0} - K\right) \right)^+, \quad \text{Ann}(s) =\sum_{i > 0} \frac{ \delta' }{(1 + \delta' s)^i }
$$

Market makers formula convention simply substitutes cash-annuity Ann for PVBP in the physically-settled case:
$$
PV_t = P_{t,T_0}  \text{Ann}(S_t) \cdot \mathbb{E}_{t}^{\text{PVBP}} \left[(\pm\left( S_{T_0} - K)\right)^+ \right], \quad S_t = \mathbb{E}_{t}^{\text{PVBP}} [S_{T_0}]
$$
which leads to a failure of put-call parity, especially when we are in IBOR multi-curve environment.

To account for the convexity correction needed, one introduces the cash-settled convexity adjusted PVBP:
$$
\text{CSCAP}(t) = B_t \cdot \mathbb{E}^Q_{t} \left[ \frac{\text{Ann}(T_0) }{B_{T_0}}  \right]
$$
and the cash-settled convexity adjusted forward:
$$
\text{CSCAF}(t) = \frac{B_t}{\text{CSCAP}(t)} \cdot \mathbb{E}^Q_{t} \left[ S_{T_0}\frac{\text{Ann}(T_0) }{B_{T_0}}  \right]
$$

In that case, we do obtain the desired put-call parity:
$$
PV_t\left[\text{payer}, K \right] - PV_t\left[\text{receiver}, K \right] = \text{CSCAP}(t) \cdot \left( \text{CSCAF}(t) - K \right)
$$


#### Exercise 3.1.1
Price via Monte-Carlo using the model from 2.2.1 a Physically-Settled ATM payer swaption with maturity 1Y for a 3M3Y IRS swap. Price via Monte-Carlo a Cash-Settled swaption with the same strike together with the convexity corrected forward CSCAP and annuity CSCAF.

### 3.2 Autocallable Caps and Floors

Given a term structure $T_0 < T_1 < \cdots < T_n$ and autocallable cap (+) or floor (-) pays at each date $T_i$, $i>0$ :
$$
\alpha \cdot \tau_i  \cdot \left( \pm \left( \mathcal{P}_{T_{i-1}, T_i} - K\right) \right)^+
$$

where $\mathcal{P}_{T_{i-1}, T_i}$ stands for a floating rate with tenor $\tau_i = T_i - T_{i-1}$ being an RFR rate (for example simple or arithmetic average daily compounding) or an IBOR rate. 

We also denote $\alpha$ the coupon rate, and $K$ the cap (resp. floor). 

The payment is conditional upon not reaching the autocallable barrier $\mathcal{P}_{T_{i-1}, T_i} < A$ (resp. $\mathcal{P}_{T_{i-1}, T_i} > A$ ), in which case the full notional is paid. At expiry date $T_n$, the notional is paid even if the barrier is not reached:
$$
\alpha \cdot \tau_n  \cdot \left( \pm \left( \mathcal{P}_{T_{n-1}, T_n} - K\right) \right)^+ + 1
$$

#### Exercise 3.2.1

Price via Monte-Carlo using the model from 2.2.1 an autocallable floor on the Key Rate starting now $T_0 = 0$ for 4 years $T_4 = 4$ with payments every year $T_i = i$ with coupon payments every year with strike $K = 0.21$, barrier $ A = 0.17$, and coupon rate $\alpha = 0.165$.


### 3.3 Target Redemption Swaps

Given a term structure $T_0 < T_1 < \cdots < T_n$, a payer (+) or receiver (-) Target Redemption Swap exchange cashflows at each date $T_i$, $i>0$:

$$
C_i = \pm \tau_i \cdot \left( \mathcal{P}_{T_{i-1}, T_i} - K  \right) 
$$

A Target Redemption Note pays:
$$
C_i = \tau_i \cdot \left( \pm \left(\mathcal{P}_{T_{i-1}, T_i} - K \right) \right)^+
$$

As usual, the floating leg $\mathcal{P}_{T_{i-1}, T_i}$ stands for an RFR or IBOR rate over $[T_{i-1}, T_i]$. 

The payments are conditional upon not reaching a cumulative return target:
$$
\sum_{j=1}^i C_j < B
$$

If the barrier is reached payments are stopped and the contract expires.


#### Exercise 3.3.1

Compute the present value of a receiver IRS swap starting now $T_0$ for 4 years $T_4 = 4$ with payments every year $T_i = i$ and a fixed leg $K = 0.21$ if the floating leg was a Key Rate staying exactly at $0.17$ over all those 4 years (don't forget the arithmetic average daily compounding). 

Use this value as cumulative return target $B$ to price via Monte-Carlo using the model from 2.2.1 a receiver Target Redemption Swap over the Key Rate with strike $K = 0.21$ and the above term structure $T_{0,\dots, 4}$. 


### 3.4 Constant Maturity Swap

Given a term structure $T_0 < T_1 < \cdots < T_n$, a payer (+) or receiver (-) Constant Maturity Swap exchange cashflows at each date $T_i$, $i>0$:
$$
\pm \tau_i \cdot \left( S_{T_i} \left[ \mathcal{T}  \right] - K  \right) 
$$

where $\mathcal{T}$ is a rolling term structure.

#### Exercise 3.4.1

Price via Monte-Carlo using the model 2.2.1, a receiver constant maturity swap on the Key Rate with term structure $T_i = i$, with $i = 1, \dots, 4$ with rolling term structure 3M1Y. 

### 3.5 Callable Swap

A callable swap is an IRS swap with the option to terminate the contract at each time payment time $T_i$, $i>0$. Given a term structure $T_0 < T_1 < \cdots < T_n$, the payoff at time $T_i$, $i>0$ if executed:
$$
\left(  \left( K - S_{T_i} \left[ T_i, T_n \right] \right)^+  + S_{T_i} \left[ T_i, T_n \right] - K  \right) \sum_{j=i+1}^n \tau_j P_{T_i, T_j} = \left( K \sum_{j=i+1}^n \tau_j P_{T_i, T_j} - 1 + P_{T_i,T_n} \right)^+ + 1 - P_{T_i,T_n} - K \sum_{j=i+1}^n \tau_j P_{T_i, T_j} 
$$

The optionality leg:
$$
\left( K \sum_{j=i+1}^n \tau_j P_{T_i, T_j} - 1 + P_{T_i,T_n} \right)^+
$$

is priced using dynamic programming. 

#### Exercise 3.5.1

Using American Monte-Carlo for the optionality leg, price a callable swap on the Key Rate with term structure $T_i = i$, with $i=0,\dots,4$, option to cancel at $i>0$ and strike $K = 0.21$.